In [3]:
import sys
import os
from tqdm import tqdm, trange
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import json
from Models.AutoEncoder import *
from utils.utils import *
from sklearn.metrics import f1_score
import numpy as np
from dotenv import load_dotenv
import time  # 파일 상단에 import 추가
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
from time import time

In [17]:
device = torch.device('mps' if torch.mps.is_available() else 'cpu')

In [ ]:
num_epochs = 100
batch_size = 128
lr = 1e-4

In [4]:
# 데이터 준비
cat_features = [
    'Gender',
    'Zipcode',
    'Day',
    'Card Brand',
    'Card Type',
    'Has Chip',
    'Whether Security Chip is Used',
    'Error Message',
    'WeekDay',
    'Credit Signal',
    'PIN Change',
    'Security Level'
]
num_features = [
    'Current Age',
    'Retirement Age',
    'Per Capita Income - Zipcode',
    'Yearly Income',
    'Total Debt',
    'Credit Score',
    'Valid Month',
    'Credit Limit',
    'Since Open Month',
    'Year PIN last Changed',
    'Amount',
    'Credit Util',
    'Years Changed PIN',
    'Security Score'
]
discarded = [
    'User',
    'Birth Year',
    'Birth Month',
    'Year',
    'Month',
    'Merchandise Code',
    'Card',
    'Card Number',
    'Expires',
    'Acct Open Date',
]

In [5]:
data_path = 'Data/[24-2 DS_Project2] Data.csv'

In [9]:
(train_cat_X, train_num_X, train_y), (valid_cat_X, valid_num_X, valid_y), _ , _ = dt_process_data(
        '../Data/[24-2 DS_Project2] Data.csv', 
        cat_features, 
        num_features,
        discarded
    )

TRANSITION
IQR
SPLIT
DISCARD
SCALE
ENCODE
UNLABEL
TARGET
TRAIN CAT/NUM
VALID CAT/NUM
RETURN


In [18]:
checkpoint = torch.load('../Models/best_autoencoder.pth')
model.load_state_dict(checkpoint['model_state_dict'])

/var/folders/52/2pbbt5ss6d584yv_994fmyz40000gn/T/ipykernel_68297/3500526499.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('../Models/best_autoe

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [8]:
#train_dataset = AE_Dataset(train_cat_X, train_num_X, train_y, device)
valid_dataset = AE_Dataset(valid_cat_X, valid_num_X, valid_y, device)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

NameError: name 'train_y' is not defined

In [ ]:
model = AutoEncoder(encoding_dim=32, cat_features=cat_features, num_features=num_features).to(device)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
criterion = nn.MSELoss()

In [ ]:
latent_variables = model.get_embedding(
    torch.tensor(train_cat_X.values, dtype=torch.long).to(device),
    torch.tensor(train_num_X.values, dtype=torch.float).to(device),
)

In [ ]:
def visualize_latent_space_tsne(latent_variables, perplexity=30, n_iter=1000, save_path=None):
    """
    t-SNE를 사용하여 잠재 공간을 시각화합니다.
    """
    Z = latent_variables
    
    # t-SNE 수행
    print(f"Running t-SNE on {Z.shape[1]}-dimensional data...")
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
    Z_tsne = tsne.fit_transform(Z)

    # 시각화
    plt.figure(figsize=(15, 10))

    # t-SNE 산점도
    plt.subplot(1, 2, 1)
    scatter = plt.scatter(Z_tsne[:, 0], Z_tsne[:, 1], c=Z_tsne[:, 0],
                          cmap='viridis', alpha=0.6)
    plt.colorbar(scatter)
    plt.title('t-SNE Visualization of Latent Space\n(colored by first component)')
    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')

    # 밀도 플롯
    plt.subplot(1, 2, 2)
    sns.kdeplot(data=pd.DataFrame(Z_tsne, columns=['t-SNE 1', 't-SNE 2']),
                x='t-SNE 1', y='t-SNE 2', cmap='viridis', fill=True)
    plt.title('Density Plot of t-SNE Embeddings')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

    plt.show()

def analyze_latent_space(model, data, scaler):
    """
    잠재 공간의 통계적 특성을 분석합니다.
    """
    # 잠재 변수 추출
    scaled_data = scaler.transform(data)
    X = torch.FloatTensor(scaled_data)

    model.eval()
    with torch.no_grad():
        _, Z = model(X)
        Z = Z.numpy()

    # 기본 통계량 계산
    stats_df = pd.DataFrame({
        'mean': Z.mean(axis=0),
        'std': Z.std(axis=0),
        'min': Z.min(axis=0),
        'max': Z.max(axis=0)
    })

    print("\nLatent Space Statistics:")
    print(stats_df)

    # 상관관계 분석
    plt.figure(figsize=(10, 8))
    sns.heatmap(pd.DataFrame(Z).corr(), annot=True, cmap='coolwarm', center=0)
    plt.title('Correlation Matrix of Latent Variables')
    plt.tight_layout()
    plt.show()

In [ ]:
def main():

    # t-SNE 시각화
    visualize_latent_space_tsne(latent_variables= latent_variables,
                                perplexity=30,
                                n_iter=1000,
                                save_path='tsne_visualization.png')

    # 잠재 공간 분석
    #analyze_latent_space(model, data, scaler)

if __name__ == "__main__":
    main()